In [2]:
import os
import scipy.io
import numpy as np
from tqdm import tqdm

In [3]:
src = "/home/george-vengrovski/Documents/lab_canary_data"
dst = "/home/george-vengrovski/Documents/canary_song_detector/data"
format = ".mat"

In [4]:
src_files = os.listdir(src)

def mat_to_npy(src, dst, filename, spec_height=513):
    """
    Params:
    - file dir
    - 
    """
    mat_data = scipy.io.loadmat(src)
    mat_data = mat_data["song_data"]
    mat_data = mat_data[0][0]

    arr1 = mat_data[0]
    arr2 = mat_data[1]

    # beware if spec shape changes, this might cause error
    if arr1.shape[0] == spec_height:
        spec = arr1
        raw_labels = arr2.astype(int)
    else:
        spec = arr2
        raw_labels = arr1.astype(int)

    if raw_labels.shape == (0,0):
        song = False
    else:
        song = True

    # labels will be the same length as the song, but it will be filled with 1s between indcies of start and stops
    labels = np.zeros((spec.shape[1],))
    if song:
        num_entries = raw_labels.shape[0]
        for i in range(num_entries):
            if i % 2 == 0:
                start = raw_labels[i].item()
                stop = raw_labels[i + 1].item()
                labels[start:stop] = 1  # Using NumPy's slicing
    
    spec = spec.reshape(1, spec_height, -1)
    # Save spec, labels, and song as .npz file

    save_name = os.path.join(dst, filename)
    np.savez(save_name, s=spec, labels=labels, song=song)
    
if format == ".mat":
    for bird in src_files:
        for day in os.listdir(os.path.join(src, bird)): 
            d = os.path.join(src, bird, day)
            for file in tqdm(os.listdir(d), desc=f"Processing files for bird {bird} on day {day}"):
                npy_file = mat_to_npy(src = os.path.join(src, bird, day, file), dst= dst, filename= file)
        
                


Processing files for bird USA5207 on day joined_trained:   0%|          | 0/5929 [00:00<?, ?it/s]

Processing files for bird USA5207 on day joined_trained: 100%|██████████| 5929/5929 [02:13<00:00, 44.37it/s]
Processing files for bird USA5199 on day joined_test:  67%|██████▋   | 5246/7791 [01:11<00:37, 68.36it/s]